# Object Detection using Tensorflow Object Detection API

In this lab exercise, we will use our trained custom object detection model to detect balloons in both images and videos. 


## Imports and Setup

Let's start with the base imports.

In [4]:
!pip install opencv-python

  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)


In [16]:
!pip install tensorflow-object-detection

  Using cached tensorflow_object_detection-0.0.6-py3-none-any.whl
  Using cached pycocotools-fix-2.0.0.9.tar.gz (124 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pycocotools-fix
Failed to build pycocotools-fix
  Running setup.py install for pycocotools-fix: started
  Running setup.py install for pycocotools-fix: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [20 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\coco.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\cocoeval.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\mask.py -> build\lib.win-amd64-cpython-310\pycocotools
  copying pycocotools\__init__.py -> build\lib.win-amd64-cpython-310\pycocotools
  running build_ext
  skipping 'pycocotools\_mask.c' Cython extension (up-to-date)
  building 'pycocotools._mask' extension
  creating build\temp.win-amd64-cpython-310
  creating build\temp.win-amd64-cpython-310\Release
  creating build\temp.win-amd64-cpython-310\Release\common
  creating build\temp.win-amd64-cpython-310\Release\pycocotools
  "C:\Program Files (x86)\Micro

In [1]:
!python --version

Python 3.9.16


In [15]:
!pip list

Package                        Version
------------------------------ ---------------
absl-py                        1.3.0
aiohttp                        3.8.3
aiosignal                      1.2.0
alabaster                      0.7.12
anaconda-client                1.11.1
anaconda-navigator             2.4.1
anaconda-project               0.11.1
anyio                          3.5.0
appdirs                        1.4.4
argon2-cffi                    21.3.0
argon2-cffi-bindings           21.2.0
arrow                          1.2.3
asgiref                        3.7.2
astroid                        2.14.2
astropy                        5.1
asttokens                      2.0.5
astunparse                     1.6.3
async-generator                1.10
async-timeout                  4.0.2
atomicwrites                   1.4.0
attr                           0.3.1
attrs                          22.1.0
Automat                        20.2.0
autopep8                       1.6.0
azure-core           

In [8]:
!pip install object-detection

     ---------------------------------------- 1.5/1.5 MB 13.6 MB/s eta 0:00:00
  Using cached Cython-0.29.35-py2.py3-none-any.whl (988 kB)
  Using cached contextlib2-21.6.0-py2.py3-none-any.whl (13 kB)
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
     ------------------------------------- 895.7/895.7 kB 27.7 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 23.4 MB/s eta 0:00:00
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
label-studio 1.7.3 requires python-dateutil==2.8.1, but you have python-dateutil 2.8.2 which is incompatible.
google-api-core 2.10.0 requires protobuf<5.0.0dev,>=3.20.1, but you have protobuf 3.19.6 which is incompatible.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.28.0 which is incompatible.
anaconda-client 1.11.1 requires pytz>=2021.3, but you have pytz 2019.3 which is incompatible.


In [5]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from six import BytesIO
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import cv2

## Utilities

Run the following cell to create some utils that will be needed later:

- Helper method to load an image


In [6]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: the file path to the image

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    image = None
    if(path.startswith('http')):
        response = urlopen(path)
        image_data = response.read()
        image_data = BytesIO(image_data)
        image = Image.open(image_data)
    else:
        image_data = tf.io.gfile.GFile(path, 'rb').read()
        image = Image.open(BytesIO(image_data))

        (im_width, im_height) = image.size
        
    return np.array(image.getdata())[:,:3].reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)



## Visualization tools

To visualize the images with the proper detected boxes, we will use the TensorFlow Object Detection API. 

In [21]:
!pip install --upgrade object_detection

In [38]:
!conda activate ataTry

In [2]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = "C:/Users/HP/Desktop/y3s1/aap/ataTry/project/balloon_project/exported_models/my_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (C:\Users\HP\anaconda3\envs\ataTry\lib\site-packages\object_detection\protos\__init__.py)

In [17]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

ImportError: cannot import name 'string_int_label_map_pb2' from 'object_detection.protos' (C:\Users\HP\anaconda3\lib\site-packages\object_detection\protos\__init__.py)

### Load label map data (for plotting).

Label maps correspond index numbers to category name. For example, if our COCO-trained model predicts `5`, we know that this corresponds to `airplane`. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

We will load the label_map we created for our balloon detector, which only contains one class.

In [ ]:
PATH_TO_LABELS = 'data/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
category_index

## Loading our saved custom model

Here we will load our trained balloon detection model and use it later to detect our picture and videos.

In [ ]:
## Uncomment the following if you want to use a pretrained balloon detection model

# !wget https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/pretrained-models/balloon_model.tar.gz
# !mkdir mymodel
# !tar xzvf balloon_model.tar.gz -C mymodel

In [ ]:
model = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
experiment='run1'
export_root_dir = '/home/ubuntu/balloon_project/exported_models'

# Uncomment the following if you want to use a pretrained balloon detection model
# model_dir = 'mymodel/saved_model'
model_dir = os.path.join(export_root_dir, f'{model}/{experiment}/saved_model')

# load the model
detection_model = tf.saved_model.load(str(model_dir))

## Loading an image

Let's try the model on your own image. You can upload your image using the code below. The code below uses ipywidgets [FileUpload](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) widget to upload file


In [ ]:
import ipywidgets as widgets

uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

display(uploader)


After upload, we need to write the upload data to a file. The content of the upload data can be found in ``uploader.data[0]``.

In [ ]:
key = next(iter(uploader.value))

samples_folder = 'test_samples'

if not os.path.exists(samples_folder):
    os.mkdir(samples_folder)

image_path = os.path.join(samples_folder, key)

with open(image_path, "w+b") as file:
    file.write(uploader.data[0])



In [ ]:
image_np = load_image_into_numpy_array(image_path)
plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()

## Doing the inference

To do the inference we just need to call our detection model. The detection model expect the following input and output a dictionary of items described below:

*Inputs*:

A three-channel image of variable size - the model does NOT support batching. The input tensor is a tf.uint8 tensor with shape [1, height, width, 3] with values in [0, 255].

*Outputs*:

The output dictionary contains:

- num_detections: a tf.int tensor with only one value, the number of detections [N].

- detection_boxes: a tf.float32 tensor of shape [N, 4] containing bounding box coordinates in the following order: [ymin, xmin, ymax, xmax].
- detection_classes: a tf.int tensor of shape [N] containing detection class index from the label file.
- detection_scores: a tf.float32 tensor of shape [N] containing detection scores.
- raw_detection_boxes: a tf.float32 tensor of shape [1, M, 4] containing decoded detection boxes without Non-Max suppression. M is the number of raw detections.
- raw_detection_scores: a tf.float32 tensor of shape [1, M, 90] and contains class score logits for raw detection boxes. M is the number of raw detections.
- detection_anchor_indices: a tf.float32 tensor of shape [N] and contains the anchor indices of the detections after NMS.
- detection_multiclass_scores: a tf.float32 tensor of shape [1, N, 90] and contains class score distribution (including background) for detection boxes in the image including background class.


In [ ]:
# running inference
results = detection_model(image_np)

# different object detection models have additional results
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

## Visualizing the results

Here is where we will need the TensorFlow Object Detection API to show the squares from the inference step (and the keypoints when available). The full documentation of this method can be seen [here](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py)

Here you can, for example, set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [ ]:
label_id_offset = 0
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.60,
      agnostic_mode=False,
      line_thickness=2)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_detections[0])
plt.show()

## Object Detection on Video

We can also do object detection on a video file. We just treat our video file as a sequence of frames (images) and we run the detector on each frame. After we draw the bounding boxes on the frame (image), we then write the new image (with bounding boxes) to a video file, frame by frame. 


In [ ]:
def write_video(video_in_filepath, video_out_filepath, detection_model):
    if not os.path.exists(video_in_filepath):
        print('video filepath not valid')
    
    video_reader = cv2.VideoCapture(video_in_filepath)
    
    nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = video_reader.get(cv2.CAP_PROP_FPS)
    
    video_writer = cv2.VideoWriter(video_out_filepath,
                               cv2.VideoWriter_fourcc(*'mp4v'), 
                               fps, 
                               (frame_w, frame_h))

    for i in tqdm(range(nb_frames)):
        ret, image_np = video_reader.read()
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.uint8)
        results = detection_model(input_tensor)
        viz_utils.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  results['detection_boxes'][0].numpy(),
                  (results['detection_classes'][0].numpy()+ label_id_offset).astype(int),
                  results['detection_scores'][0].numpy(),
                  category_index,
                  use_normalized_coordinates=True,
                  max_boxes_to_draw=200,
                  min_score_thresh=.50,
                  agnostic_mode=False,
                  line_thickness=2)

        video_writer.write(np.uint8(image_np))
                
    # Release camera and close windows
    video_reader.release()
    video_writer.release() 
    cv2.destroyAllWindows() 
    cv2.waitKey(1)

In [ ]:
uploader = widgets.FileUpload(
    accept='.mp4',  # Accepted file extension only '.mp4'
    multiple=False  # True to accept multiple files upload else False
)

display(uploader)

In [ ]:
key = next(iter(uploader.value))

samples_folder = 'test_samples'

if not os.path.exists(samples_folder):
    os.mkdir(samples_folder)
video_in_file = os.path.join(samples_folder, key)
with open(video_in_file, "w+b") as file:
    file.write(uploader.data[0])

In [ ]:
from pathlib import Path

basename = Path(video_in_file).stem
video_out_file = os.path.join(samples_folder, basename + '_detected' + '.mp4')

write_video(video_in_file, video_out_file, detection_model)